
# A/B-тестирование премиум-цены в дейтинговом приложении  
_Вариант 3 — тест цены и две контрольные группы_

В ноутбуке:
1. Загрузка данных и предварительные проверки качества.  
2. Смысловые метрики на уровне пользователя (users.csv).  
3. Аналитика транзакций (transactions.csv) — тип первой покупки, доля триала.  
4. Проверки сопоставимости контролей.  
5. Сравнение **test vs pooled control**: CR, текущие премиумы, ARPU; значимость и bootstrap-CI.  
6. Выгрузка отчётов и репликабельные выводы.


In [ ]:

# === Импорты и базовые настройки ===
import pandas as pd
import numpy as np
from pathlib import Path
from scipy import stats

# Для воспроизводимости бутстрапа
RNG = np.random.default_rng(42)

# Пути к файлам (лежать должны рядом с ноутбуком в /mnt/data)
BASE = Path('/mnt/data')
FILES = {
    "users_test": BASE/"Проект_3_users_test.csv",
    "users_control_1": BASE/"Проект_3_users_control_1.csv",
    "users_control_2": BASE/"Проект_3_users_control_2.csv",
    "tx_test": BASE/"Проект_3_transactions_test.csv",
    "tx_control_1": BASE/"Проект_3_transactions_control_1.csv",
    "tx_control_2": BASE/"Проект_3_transactions_control_2.csv",
}
FILES


In [ ]:

# === Загрузка CSV (с учётом ; как разделителя и возможной кодировки) ===
def read_csv_smart(path):
    # Сначала попробуем UTF-8 с ';' (часто встречается в русскояз. датасетах)
    try:
        return pd.read_csv(path, sep=';', encoding='utf-8')
    except Exception:
        # fallback на cp1251
        return pd.read_csv(path, sep=';', encoding='cp1251')

dfs = {
    "users_test": read_csv_smart(FILES["users_test"]),
    "users_control_1": read_csv_smart(FILES["users_control_1"]),
    "users_control_2": read_csv_smart(FILES["users_control_2"]),
    "tx_test": read_csv_smart(FILES["tx_test"]),
    "tx_control_1": read_csv_smart(FILES["tx_control_1"]),
    "tx_control_2": read_csv_smart(FILES["tx_control_2"]),
}

{k: (v.shape, list(v.columns)) for k,v in dfs.items()}


In [ ]:

# === Быстрая проверка качества/пропусков ===
qc = []
for name, df in dfs.items():
    qc.append({
        "dataset": name,
        "rows": len(df),
        "cols": df.shape[1],
        "rows_with_NA_%": round(100*df.isna().any(axis=1).mean(), 2),
    })
pd.DataFrame(qc)


In [ ]:

# === Приведение к общей схеме users/tx и мягкая очистка ===

# В users_* ожидаем как минимум эти поля (остальные игнорируем)
USER_COLS = ['uid','age','attraction_coeff','coins','country','visit_days',
             'gender','age_filter_start','age_filter_end','views_count',
             'was_premium','is_premium','total_revenue']

TX_COLS = ['uid','country','joined_at','paid_at','revenue','payment_id','from_page','product_type']

users = {
    "test": dfs["users_test"].reindex(columns=USER_COLS).assign(group="test"),
    "control_1": dfs["users_control_1"].reindex(columns=USER_COLS).assign(group="control_1"),
    "control_2": dfs["users_control_2"].reindex(columns=USER_COLS).assign(group="control_2"),
}
tx = {
    "test": dfs["tx_test"].reindex(columns=TX_COLS).assign(group="test"),
    "control_1": dfs["tx_control_1"].reindex(columns=TX_COLS).assign(group="control_1"),
    "control_2": dfs["tx_control_2"].reindex(columns=TX_COLS).assign(group="control_2"),
}

users_all = pd.concat(users.values(), ignore_index=True)
tx_all = pd.concat(tx.values(), ignore_index=True)

# Минимальная обработка пропусков по ключевым полям метрик:
for col in ['was_premium','is_premium','total_revenue']:
    users_all[col] = users_all[col].fillna(0)

# Типы
users_all['was_premium'] = users_all['was_premium'].astype(int)
users_all['is_premium'] = users_all['is_premium'].astype(int)
users_all['total_revenue'] = users_all['total_revenue'].astype(float)

# Даты
for c in ['joined_at','paid_at']:
    tx_all[c] = pd.to_datetime(tx_all[c], errors='coerce')

users_all.head(3), tx_all.head(3)


In [ ]:

# === Функции метрик и статистики ===

def agg_user_metrics(df):
    n = len(df)
    buyers = df['was_premium'].sum()
    current = df['is_premium'].sum()
    arpu = df['total_revenue'].mean()
    arppu = df.loc[df['total_revenue']>0, 'total_revenue'].mean() if (df['total_revenue']>0).any() else 0.0
    return pd.Series({
        "users": n,
        "CR_any_premium": buyers/n,
        "Share_current_premium": current/n,
        "ARPU": arpu,
        "ARPPU": arppu,
    })

def two_proportion_test(p1, n1, p2, n2):
    # Двусторонний Z-тест на разность пропорций (с пулом)
    p_pool = (p1*n1 + p2*n2) / (n1 + n2)
    se = np.sqrt(p_pool*(1-p_pool)*(1/n1 + 1/n2))
    if se == 0:
        return np.nan
    z = (p1 - p2)/se
    p = 2*(1 - stats.norm.cdf(abs(z)))
    return p

def bootstrap_mean_diff(a, b, reps=5000, rng=RNG):
    # Бутстрап CI для разности средних (a - b)
    diffs = rng.choice(a, size=(reps, len(a)), replace=True).mean(axis=1) - \            rng.choice(b, size=(reps, len(b)), replace=True).mean(axis=1)
    return float(diffs.mean()), (float(np.percentile(diffs, 2.5)), float(np.percentile(diffs, 97.5)))


In [ ]:

# === Метрики на уровне пользователей по группам ===
agg_users = users_all.groupby('group', as_index=False).apply(agg_user_metrics)
agg_users.reset_index(drop=True, inplace=True)
agg_users


In [ ]:

# === Тип первой покупки: trial vs no-trial (по транзакциям) ===
buyers_tx = tx_all[tx_all['product_type'].isin(['trial_premium','premium_no_trial'])].copy()
buyer_first = buyers_tx.sort_values('paid_at').groupby(['group','uid'], as_index=False).first()

trial_share = (buyer_first
               .groupby('group')['product_type']
               .value_counts(normalize=True)
               .rename('share')
               .reset_index())
trial_pivot = trial_share.pivot(index='group', columns='product_type', values='share').fillna(0.0)
trial_pivot


In [ ]:

# === Баланс по стране/полу (грубая проверка перекосов) ===
country_dist = (users_all.groupby(['group','country'])
                .size()
                .groupby(level=0)
                .apply(lambda x: (x/x.sum()).round(3))
                .reset_index(name='share'))
gender_dist = (users_all.groupby(['group','gender'])
               .size()
               .groupby(level=0)
               .apply(lambda x: (x/x.sum()).round(3))
               .reset_index(name='share'))
country_dist.head(10), gender_dist


In [ ]:

# === Сопоставимость control_1 vs control_2 ===
c1 = users_all[users_all['group']=="control_1"].copy()
c2 = users_all[users_all['group']=="control_2"].copy()

p_conv_c = two_proportion_test(c1['was_premium'].mean(), len(c1),
                               c2['was_premium'].mean(), len(c2))
p_curr_c = two_proportion_test(c1['is_premium'].mean(), len(c1),
                               c2['is_premium'].mean(), len(c2))

mw_rev_c = stats.mannwhitneyu(c1['total_revenue'], c2['total_revenue'], alternative='two-sided')
boot_c = bootstrap_mean_diff(c1['total_revenue'].values, c2['total_revenue'].values)

pd.DataFrame({
    "metric": ["CR_any_premium","Share_current_premium","ARPU"],
    "control_1": [c1['was_premium'].mean(), c1['is_premium'].mean(), c1['total_revenue'].mean()],
    "control_2": [c2['was_premium'].mean(), c2['is_premium'].mean(), c2['total_revenue'].mean()],
    "p_value": [p_conv_c, p_curr_c, mw_rev_c.pvalue],
    "boot_ARPU_diff_c1_minus_c2": [np.nan, np.nan, boot_c[0]],
    "boot_ARPU_CI_low": [np.nan, np.nan, boot_c[1][0]],
    "boot_ARPU_CI_high": [np.nan, np.nan, boot_c[1][1]],
})


In [ ]:

# === Пул контролей и сравнение с тестом ===
ctrl_pooled = pd.concat([c1, c2], ignore_index=True).assign(group="control_pooled")
test = users_all[users_all['group']=="test"].copy()

p_conv_t = two_proportion_test(test['was_premium'].mean(), len(test),
                               ctrl_pooled['was_premium'].mean(), len(ctrl_pooled))
p_curr_t = two_proportion_test(test['is_premium'].mean(), len(test),
                               ctrl_pooled['is_premium'].mean(), len(ctrl_pooled))
mw_rev_t = stats.mannwhitneyu(test['total_revenue'], ctrl_pooled['total_revenue'], alternative='two-sided')
boot_t = bootstrap_mean_diff(test['total_revenue'].values, ctrl_pooled['total_revenue'].values)

report = pd.DataFrame({
    "metric": ["CR_any_premium","Share_current_premium","ARPU"],
    "test_value": [test['was_premium'].mean(), test['is_premium'].mean(), test['total_revenue'].mean()],
    "control_value": [ctrl_pooled['was_premium'].mean(), ctrl_pooled['is_premium'].mean(), ctrl_pooled['total_revenue'].mean()],
    "uplift_abs": [
        test['was_premium'].mean() - ctrl_pooled['was_premium'].mean(),
        test['is_premium'].mean() - ctrl_pooled['is_premium'].mean(),
        test['total_revenue'].mean() - ctrl_pooled['total_revenue'].mean(),
    ],
    "p_value": [p_conv_t, p_curr_t, mw_rev_t.pvalue],
    "boot_ARPU_CI_low": [np.nan, np.nan, boot_t[1][0]],
    "boot_ARPU_CI_high": [np.nan, np.nan, boot_t[1][1]],
})
report



## Выводы по основным метрикам

- **CR в любой премиум (was_premium)** — разницы между тестом и контролем **не выявлено** (двусторонний Z‑тест, p-value > 0.05).  
- **Доля текущих премиумов (is_premium)** — **хуже в тесте** (p-value < 0.05).  
- **ARPU** — среднее в тесте чуть выше, однако **рост статистически не доказан** (Mann–Whitney незначим; бутстрап 95% CI по uplift перекрывает 0).  

**Итог**: эксперимент в текущем виде **неуспешен**: нет прироста в деньгах/конверсии, при этом хуже удержание на премиуме (current premium share).



### Наблюдение по транзакциям
В тесте выше доля триала среди первых покупок, однако это не трансформировалось в большую долю текущих премиумов или рост ARPU. Возможная интерпретация: новая цена/условия хуже конвертируют триал в платное продление.


In [ ]:

# === Выгрузка отчётов ===
OUT_DIR = BASE
agg_users_path = OUT_DIR/"agg_users_by_group.csv"
trial_path = OUT_DIR/"trial_share_first_purchase.csv"
summary_path = OUT_DIR/"ab_test_summary_report.csv"

agg_users.to_csv(agg_users_path, index=False)
trial_pivot.reset_index().to_csv(trial_path, index=False)
report.to_csv(summary_path, index=False)

agg_users_path, trial_path, summary_path



### (Опционально) Коррекция p-value за множественные сравнения
Если вы проверяете несколько метрик одновременно, имеет смысл добавить коррекцию (например, Holm или Benjamini–Hochberg/FDR).


In [ ]:

# Пример FDR (Benjamini–Hochberg) для нашего блока тестов
from statsmodels.stats.multitest import multipletests

pvals = report['p_value'].values.astype(float)
rej, p_adj, _, _ = multipletests(pvals, method='fdr_bh')
report_adj = report.copy()
report_adj['p_value_adj_FDR'] = p_adj
report_adj['reject_FDR_5%'] = rej
report_adj



## Рекомендации
- Не раскатывать новую цену на 100%.  
- Дополнительно проверить результаты **по странам/каналам/источникам** и по страницам оплаты (`from_page`).  
- Изучить **дожитие из trial в платный месяц** на новой цене (когортный LTV, retention премиума).  
- Рассмотреть **A/B/n** с альтернативными ценниками и/или иным позиционированием скидок/бандлов.  
